# 🧑‍⚖️ LLM-as-a-judge: Advanced Retrieval Evaluation

이 노트북은 단순 키워드 매칭(Hit@k)을 넘어, **LLM(GPT-4o)**을 사용하여 리트리버의 실제 성능을 정성적으로 평가합니다.

### 🚀 주요 프로세스
1.  **Hard Query Augmentation**: 기존의 단순한 질문을 실제 유저들이 할 법한 모호하고 복잡한 질문으로 변환합니다.
2.  **Hybrid Retrieval**: 변환된 질문으로 검색을 수행합니다.
3.  **LLM Scoring**: 검색 결과가 질문의 의도에 얼마나 부합하는지 1~5점 척도로 채점합니다.

In [ ]:
# ✅ 실험 파라미터
SAMPLE_SIZE = 10        # 비용 문제로 적은 샘플로 시작 권장
JUDGE_MODEL = "gpt-4o"  # 평가자 모델
RETRIEVER_K = 5         # 검색할 문서 수

# 환경 설정
import sys
import os
from dotenv import load_dotenv
import pandas as pd
import json
import random
import asyncio
from typing import List, Optional
from pydantic import BaseModel, Field

# LangChain
from langchain.chat_models import init_chat_model
from src.core.config import LLMConfig
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

load_dotenv(os.path.join(project_root, ".env"))
print(f"Project Root: {project_root}")

In [ ]:
# Initialize Components
from src.retrieval.hybrid_search import HybridRetriever

retriever = HybridRetriever(version="v3", collection_name="care_guides")
llm_judge = init_chat_model(JUDGE_MODEL, model_provider="openai", temperature=0)

print("✅ Components Initialized.")

## 1. 데이터셋 로드 및 샘플링

In [ ]:
dataset_path = os.path.join(project_root, "data/v3/golden_dataset.json")

if os.path.exists(dataset_path):
    with open(dataset_path, "r", encoding="utf-8") as f:
        FULL_DATASET = json.load(f)
    print(f"📚 Loaded {len(FULL_DATASET)} test cases.")
else:
    print("⚠️ Dataset not found. Using mock.")
    FULL_DATASET = [{"query": "메인쿤 특징", "expected_keyword": "메인 쿤"}]

test_dataset = random.sample(FULL_DATASET, min(SAMPLE_SIZE, len(FULL_DATASET)))
print(f"🧪 Selected {len(test_dataset)} samples for evaluation.")

## 2. Hard Query Generation (Augmentation)
단순한 키워드형 질문을 문맥이 포함된 복합 질문으로 변환합니다.
- **Before**: "메인쿤 특징"
- **After**: "덩치가 크고 털이 풍성한 고양이를 키우고 싶은데, 성격이 강아지처럼 온순한 품종이 뭐가 있을까?"

In [ ]:
class AugmentedQuery(BaseModel):
    hard_query: str = Field(..., description="The transformed complex query")

augment_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative user helper. Convert the given simple query into a more natural, complex, and slightly ambiguous user question that maintains the original intent but is harder for a keyword search engine to match. Do NOT include the specific target keyword if possible, describe its characteristics instead."),
    ("human", "Original Query: {query}\nTarget Keyword: {target}")
])

augment_chain = augment_prompt | llm_judge.with_structured_output(AugmentedQuery)

async def generate_hard_queries(dataset):
    augmented_dataset = []
    print("🚀 Generating Hard Queries...")
    
    for item in dataset:
        original_query = item.get("query")
        target = item.get("expected_keyword", item.get("target"))
        
        try:
            res = await augment_chain.ainvoke({"query": original_query, "target": target})
            augmented_dataset.append({
                "original_query": original_query,
                "hard_query": res.hard_query,
                "target": target
            })
            print(f" - {original_query} -> {res.hard_query}")
        except Exception as e:
            print(f"❌ Error generating query for {original_query}: {e}")
            
    return augmented_dataset

# 실행
augmented_data = await generate_hard_queries(test_dataset)

## 3. Retrieval & LLM Evaluation
생성된 Hard Query로 검색을 수행하고, 결과를 LLM이 채점합니다.

In [ ]:
# 평가 결과 모델
class JudgeResult(BaseModel):
    score: int = Field(..., description="Score from 1 to 5")
    reasoning: str = Field(..., description="Reasoning for the score")

# 평가 프롬프트
judge_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert search relevance evaluator.
Apply the following score rubric to the retrieved documents for the user query:

Score 5: The document provides a perfect answer to the user's implicit and explicit intent. It contains the core semantic meaning or the specific target entity described.
Score 4: The document is very relevant and helpful, but might contain slight noise or misses a minor detail.
Score 3: The document is somewhat relevant (mentions related concepts), but does not directly answer the core intent or is too broad.
Score 2: The document has very low relevance (perhaps shares a keyword) but does not help the user.
Score 1: The document is completely irrelevant.

User Query: {query}
Expected Topic/Target: {target}

Retrieved Documents:
{context}
"""),
])

judge_chain = judge_prompt | llm_judge.with_structured_output(JudgeResult)

# ==================================================================================
# 🔄 RAG Context Distillation (Same with Agent Logic)
# ----------------------------------------------------------------------------------
# 실제 에이전트(Care Team, Liaison)와 완전히 동일한 로직으로 문맥을 압축합니다.
# ==================================================================================
from langchain_core.messages import SystemMessage

llm_distill = init_chat_model(LLMConfig.ROUTER_MODEL, model_provider="openai", temperature=0)

async def distill_context(query, docs):
    if not docs:
        return ""
    
    # 1. 문서 블록 구성 (Agent와 동일 포맷)
    docs_block = "\n\n".join([
        f"[{r.get('title_refined', '') or r.get('title', '')}]\n{r.get('text', '')[:1500]}"
        for r in docs
    ])
    
    # 2. LLM 호출 (Agent와 동일 프롬프트 구조 - 단일 SystemMessage)
    messages = [
        SystemMessage(content=(
            "아래 참고 문서들에서 사용자 질문에 답하는 데 필요한 핵심 정보만 간결하게 추출하세요.\n"
            "- 불필요한 서론/반복 제거, 핵심 팩트와 수치 위주로 정리\n"
            "- 3~5개 bullet point, 총 300자 이내\n\n"
            f"[사용자 질문]\n{query}\n\n"
            f"[참고 문서]\n{docs_block}"
        ))
    ]
    
    try:
        res = await llm_distill.ainvoke(messages)
        return res.content
    except Exception as e:
        print(f"Error distilling context: {e}")
        return ""

async def run_llm_judge(aug_data):
    results = []
    print("\n⚖️ Starting LLM Evaluation...")
    
    for item in aug_data:
        query = item['hard_query']
        target = item['target']
        
        # 1. 검색
        # Specialist는 None으로 가정 (Hard Query라서 분류 어려울 수 있음)
        docs = await retriever.search(query, limit=RETRIEVER_K)
        
        # 2. Context 구성 (Distillation 적용)
        context_text = await distill_context(query, docs)
        
        # 3. 평가
        try:
            eval_res = await judge_chain.ainvoke({
                "query": query, 
                "target": target,
                "context": context_text
            })
            
            results.append({
                "Original": item['original_query'],
                "Hard_Query": query,
                "Score": eval_res.score,
                "Reasoning": eval_res.reasoning,
                "Top1_Doc": docs[0].get('title', 'No Result') if docs else 'None'
            })
            print(f"User: {query[:30]}... | Score: {eval_res.score}/5")
            
        except Exception as e:
            print(f"Error evaluating {query}: {e}")
            
    return pd.DataFrame(results)

# 평가 실행
df_eval = await run_llm_judge(augmented_data)

In [ ]:
# 결과 분석
print("="*40)
print(f"🏆 Average Score: {df_eval['Score'].mean():.2f} / 5.0")
print("="*40)
display(df_eval)